## Данные

### Обучение с учителем

В процессе взаимодействия с веб-сайтом, посетитель оставляет цифровой след, в виде последовательности URLs, которые он открывал(clickstream). Эта последовательность и будет источником features для нашего алгоритма.

Мы получили задачу обучения без учителя, которую хотелось бы свести к обучению с учителем. Но как определить метки классов? Посетители создают аккаунты на сайте или в приложениях, которые делятся на два типа: business и regular account. Отличный кандидат в метки классов: Regular = 0 и Business = 1.

В нашем DWH уже есть таблица, которая хранит сырые данные о переходах пользователей по сайту. Обозначим её как tbl_Clickstream.
Также есть таблица с аккаунтами пользователей, каждый из которых связан с таблицей визитов по некоему visitorId. Обозначим её как tbl_Accounts.

<font size="2">*На практике разработчики веб-сайтов внедряют трекинговый скрипт, который присваивает каждому посетителю уникальный ID, который кладут в cookies. Этот ID позволяет связать посетителя(visitor) с его историей посещений сайта(clickstream). По требованию GDPR сайты обязаны оповещать пользователей, что их отслеживают, поэтому мы видим всплывающие сообщения вида "This site uses cookies...". Задача привязки всех анонимных визитов к некоему пользователю или аккаунту - нетривиальная и находится за пределами обсуждения в этой статье.*</font>

In [10]:
import os
import pandas as pd

# Anonimized sample from tbl_Clickstream
pd.read_csv(os.getcwd() + r'\csv\tbl_Clickstream_cut_sample.csv').head(5)

,EventDate,EmailBy,UserName,VisitorID,URLAdjusted
0,2017-03-30 15:50:48,***@***.***,Lionel ***,1A2E63D3-722A-4694-A5BB-5925E91210A3,/install/desktop/trial-thank-you/
1,2017-06-14 10:25:19,***@***.***,Lionel ***,1A2E63D3-722A-4694-A5BB-5925E91210A3,/install/desktop/trial-thank-you/
2,2017-08-22 14:36:59,***@***.***,Lionel ***,1A2E63D3-722A-4694-A5BB-5925E91210A3,/installation/guide/
3,2017-08-24 16:14:09,***@***.***,Lionel ***,1A2E63D3-722A-4694-A5BB-5925E91210A3,/installation/guide/
4,2018-07-16 16:18:33,***@***.***,Lionel ***,1A2E63D3-722A-4694-A5BB-5925E91210A3,/buy/in-app/offer/upgrade/


In [11]:
# Anonimized sample from tbl_Accounts
pd.read_csv(os.getcwd() + r'\csv\tbl_Accounts_cut_sample.csv')

,EventDate,Email,UserName,VisitorID,IsBusiness
0,2019-08-15 16:28:23,***@***.***,Mollie ***,2F2E63F7-426A-4624-A5BB-5925D91210A3,0
1,2019-03-26 18:23:17,***@***.***,Javier ***,AE140471-F546-491B-A6C6-498DDA7FDE51,1
2,2019-03-27 02:53:58,***@***.***,Ravi ***,4F39B1E6-A638-4913-93CC-3720455F8218,0
3,2019-07-15 12:03:35,***@***.***,Viviane ***,7E56BC5A-01AE-486B-9A00-F77EE265D2CC,0
4,2019-08-16 15:36:40,***@***.***,Dennis ***,1095E018-91AF-4E65-AC85-B35734F6307E,1


###  Проблема полноты данных

На самом деле переходы по URL на сайте, это только часть комплексного user journey. А какие фичи он использовал в приложениях? А по каким элементам он кликал внутри страниц между переходами? Этой информации в нашем случае к сожалению нет. Только последовательные переходы по урлам. Посмотрим хватит ли этого для классификации.

###  Как построить пространство признаков

Мы будем группировать визиты каждого посетителя и упорядочивать их по метке времени. Тогда мы получим временной ряд визитов для каждого посетителя. Каждый визит мы можем представить в виде исмвольного токена, то есть единицей текста. 

Также мы дискретизуем временную компоненту, чтобы добавить в последовательность переходов по URL дополнительную информацию о конце сессии.

В данной задаче у нас нет трекинга времени пребывания посетителя, то есть нет длинны сессии. Поэтому мы эвристически определим какую паузу между переходами считать концом сессии. Более того, нас интесует не столько физическая сессия на сайте, сколько "логическая" сессия, в которой посетитель решал свою задачу. Например, он мог втечении драбочего дня несколько раз возвращаться к сайту. Поэтому мы добавим дополнительную эвристику, то есть поставим токен pause между событиями, которые мы посчитаем концом сессии, по выбранному нами порогу.

Введем порог 1 сутки. Это значит, что если посетитель не заходил на сайт в течении суток с последнего перехода, то он предположительно решил свою задачу. Интуиция следующая: если посетитель интересуется B2B-предложениями, то он заходит скорее всего в рабочее время. Если он в течении рабочего дня не сделал все, что он хотел, то на следующий день он вернется и продолжит. А значит суток не пройдёт. Конечно можно усложнять эвристики и учитывать другие факторы, но есть интуитивное понимание, что эта эвристика не даст какого-то существенного прироста в точности классфификации. Как минимум она удобна для того, чтобы видеть сессии, смотря на текстовое представление cliсkstream.

Clickstream каждого посетителя будет представлен в виде текстового документа: "**url_50 url_23 pause url_34 url_34 url_50 ...**"

###  Проблема качества данных 

Нужно убрать как можно больше шумов из данных, чтобы модели не "цеплялись" за них и давали в целом лучшие результаты.

Ниже примеры для нашего набора данных:

* дубликаты визитов страниц (ошибки в коде)
* аккаунты, для которых нет ни одного URL в таблице визитов (отключены куки, ошибки в коде)
* рассинхронизация часовых поясов в базах визитов и аккаунтов
* потерянные query string для GET (в нашем случае кто-то явно выключил хранение query string начиная с определенной даты)
* полностью отсутствующие параметры для POST (нет json'ов с потенциально полезной информацией)
* мусорные аккаунты с имейлами вида fff@ggg (по большей части сами веб-разработчики оставляли такое)
* аккаунты с одноразовыми имейлами
* аккаунты сотрудников компании (зарегистрированные как на рабочий, так и на личный email адрес)
* визиты на dev и stage домены (сотрудники компании)
* редкие визиты на одноразовые страницы (разработчики)
* визиты на битые урлы
* аккаунты, для которых не найдена или явно неполная история кликов 

Убираем дубликаты, применяем регулярные выражения к имейлам, синхронизируем таймзоны баз данных, отказываемся от дополнительных параметров из GET и POST, обрезаем ненужные домены, избавляемся от мусорных URL и т.д.

Идентификация этих шумов требует знаний бизнес-домена и здравого смысла. Очистка данных может занять много времени. Это неплохая иллюстрация того самого эмпирического правила "90% времени ML-проекта занимает подготовка датасета".

### Баланс классов и семплирование

Распределение двух классов(бизнес и регулярный аккаунт) в генеральной совокупности несбалансированное.  Так как модели мы будем строить вероятностные, то дисбаланс классов для нас не критичен, но все же было бы удобнее, чтобы он не был слишком большим.

Общая доля business accounts $\approx$0.6%. Применим undersamplimg, взяв каждый 20-й аккаунт для класса regular accounts.
Доля меньшего класса вырастет до $\approx$12%

### Особенности данных

Помимо однозначных шумов, вызванных ошибками, данные могут иметь особенности, которые в зависимости от задачи могут быть как шумом так и нет.

В нашем случае это:

* семантика URL (нет консистентности)
* разные урлы для одной и той же страницы (SEO-специалист сделал для каждого языка отдельный URL)
* клики по отжившим свое страницам (лендинги, промоушены, маркетинговые кампании)
* посещения одной и той же страницы много раз подряд в течении короткого времени (refresh в браузере)

При подготовке датасета необходиимо учесть и упомянутые выше особенности. 

Семантика URL нам не мешает, мы не будет их кластеризовать.  
Разные языковые версии страниц придется отловить и объединить семантически.  
Отжившие свое страницы особо не влияют на общую задачу при условии регулярного переобучения модели на свежих данных из production.  
Посещения одной и той же страницы в браузере мы устраним с помощью порога по времени.

### Построение датасета

Датасет строится следующим образом:
1. В DWH строятся data marts в виде представлений таблиц в БД, в которых отображается только нужное нам для задачи подмножество данных (фильтрация данных делается по побольшей части в OLAP DB, оптимизированной под такие задачи).
2. Код на Python выгружает отфильтрованные данные в память с помощью драйвера БД и библиотеки Pandas.
3. Производится финальная дообработка данных в Pandas, сохранение их в списки и сериализация на файловую систему с помощью библиотеки pickle.

В связи с тем, что Data Warehouse принадлежит действующей компании, SQL-скриптов и кода на Python приведено здесь не будет.